In [1]:
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

Using TensorFlow backend.


In [2]:
def word_cleaner(word):
    sp=" "
    for char in word:
#         print(char)
        if char!= "," and char!= "!" and char!= "?" and char!= "." and char!= "'" and char!= "-":
#         if char!=',':
            sp+=char
    return sp
word_cleaner("hello!")

' hello'

In [3]:
def read_glove(glove_file):
    with open(glove_file, 'r') as fi:
        words = set()
        word_to_vec_map = {}
        
        for line in fi:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [4]:
words, word_to_vec_map = read_glove('glove.6B.50d.txt')

In [5]:
print(word_to_vec_map["flabbergasted"])

[ 0.15648  -1.6418    0.03172  -0.71312   0.50644  -0.98885   0.21329
  0.535    -0.92061   0.29279  -0.12707   0.11162  -0.55497   0.057649
  0.70533   0.39483   0.32232   0.79199   0.040611 -0.25473  -0.42409
  1.215     0.79135  -0.12942   1.5903   -0.04625  -0.79736   0.93377
  0.32524  -0.41154  -0.92795   0.10712   0.43809  -0.050098 -0.81199
 -0.69166   0.72132   0.14413   0.31792  -0.016638 -0.20843  -0.61226
  0.060668  0.66585   0.15657   0.15769  -0.68858   0.48054   0.38951
  0.53648 ]


In [6]:
sentences=[]

filename = 'en.txt'
with open(filename) as fn:
    line = fn.readline()
    count = 1
    while count <2000 :
        line = fn.readline()
        if count >4:
            sentences.append(line)
        count+=1
print(sentences[100])

Western or Korean style?



In [7]:
sentences

['Exactly two years ago today, she and I buried a time capsule here.\n',
 "We promised to meet here two years later, but she hasn't come yet.\n",
 "I'm going to wait.\n",
 'Here we go.\n',
 "Please, don't move.\n",
 'One, two...\n',
 'Wait a minute.\n',
 'Hello?\n',
 'Oh, auntie.\n',
 "Sorry, I'm on my way.\n",
 "I'm really sorry.\n",
 "Yes, I'm coming.\n",
 "I'm having my photo taken.\n",
 'Bye.\n',
 'Are you ready?\n',
 'Here we go.\n',
 'One, two...\n',
 'My parents wanted a daughter, so they raised me like one.\n',
 'So I thought I was a girl until I was seven.\n',
 "I had to go to the women's public bath, too.\n",
 'The older I got,\n',
 'I thought my penis would get smaller and disappear.\n',
 'But it was the opposite.\n',
 'First Half\n',
 "He hasn't changed at all.\n",
 "No, I'm a real man now.\n",
 'Hey, asshole.\n',
 'Think clerical work in the army makes you a man?\n',
 'You irritate me!\n',
 'Give me a break, asshole.\n',
 'My job was tougher than you could imagine.\n',
 'H

In [8]:
emb_list=[]

for l in sentences:
    t=[]
    for word in l.lower().split():
        word = word_cleaner(word)
        w = word_to_vec_map.get(word)
        t.append(w)
        emb_list.append(t)

In [9]:
max_len = 25

In [10]:
batch_size = 500
context = np.zeros((500,max_len))
response = np.zeros((500,max_len))

In [11]:
vocab_size = 10000
embedding_dim = 100
max_length = 25
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type,padding="post")
print(padded[6])
print(np.shape(padded))

[124   4 528   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0]
(1995, 25)


In [12]:
index_word = {}
for word , index in word_index.items():
    index_word[index] = word

In [13]:
word_index

{'<OOV>': 1,
 'you': 2,
 'i': 3,
 'a': 4,
 'to': 5,
 'the': 6,
 'gyeon': 7,
 'woo': 8,
 'me': 9,
 'it': 10,
 "i'm": 11,
 'and': 12,
 'her': 13,
 'in': 14,
 'what': 15,
 'my': 16,
 'this': 17,
 'girl': 18,
 'is': 19,
 'your': 20,
 'are': 21,
 "don't": 22,
 'here': 23,
 "it's": 24,
 'that': 25,
 'go': 26,
 'on': 27,
 'she': 28,
 'but': 29,
 'if': 30,
 'so': 31,
 'of': 32,
 'come': 33,
 'right': 34,
 'not': 35,
 'with': 36,
 'no': 37,
 'hey': 38,
 'know': 39,
 'be': 40,
 'up': 41,
 'how': 42,
 'we': 43,
 'why': 44,
 'for': 45,
 'like': 46,
 "i'll": 47,
 'was': 48,
 'see': 49,
 "you're": 50,
 'too': 51,
 'he': 52,
 'please': 53,
 'now': 54,
 'just': 55,
 'him': 56,
 'at': 57,
 'do': 58,
 'have': 59,
 'two': 60,
 'get': 61,
 'mom': 62,
 'can': 63,
 'oh': 64,
 "she's": 65,
 "woo's": 66,
 'man': 67,
 'when': 68,
 'did': 69,
 'one': 70,
 'all': 71,
 'then': 72,
 'yes': 73,
 'over': 74,
 'there': 75,
 'wanna': 76,
 'good': 77,
 'sorry': 78,
 "what's": 79,
 'years': 80,
 'time': 81,
 'think': 82

In [14]:
index_word

{1: '<OOV>',
 2: 'you',
 3: 'i',
 4: 'a',
 5: 'to',
 6: 'the',
 7: 'gyeon',
 8: 'woo',
 9: 'me',
 10: 'it',
 11: "i'm",
 12: 'and',
 13: 'her',
 14: 'in',
 15: 'what',
 16: 'my',
 17: 'this',
 18: 'girl',
 19: 'is',
 20: 'your',
 21: 'are',
 22: "don't",
 23: 'here',
 24: "it's",
 25: 'that',
 26: 'go',
 27: 'on',
 28: 'she',
 29: 'but',
 30: 'if',
 31: 'so',
 32: 'of',
 33: 'come',
 34: 'right',
 35: 'not',
 36: 'with',
 37: 'no',
 38: 'hey',
 39: 'know',
 40: 'be',
 41: 'up',
 42: 'how',
 43: 'we',
 44: 'why',
 45: 'for',
 46: 'like',
 47: "i'll",
 48: 'was',
 49: 'see',
 50: "you're",
 51: 'too',
 52: 'he',
 53: 'please',
 54: 'now',
 55: 'just',
 56: 'him',
 57: 'at',
 58: 'do',
 59: 'have',
 60: 'two',
 61: 'get',
 62: 'mom',
 63: 'can',
 64: 'oh',
 65: "she's",
 66: "woo's",
 67: 'man',
 68: 'when',
 69: 'did',
 70: 'one',
 71: 'all',
 72: 'then',
 73: 'yes',
 74: 'over',
 75: 'there',
 76: 'wanna',
 77: 'good',
 78: 'sorry',
 79: "what's",
 80: 'years',
 81: 'time',
 82: 'think'

In [15]:
index=0
for idx in range(0,950,2):
    context[index,:] = padded[idx]
    response[index,:] = padded[idx+1]
    index +=1
    
print(np.shape(context))
print(context[6])
print(response[0])
response = list(response.swapaxes(0,1))

(500, 25)
[ 11. 309.  16. 417. 418.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[ 43. 415.   5.  89.  23.  60.  80. 161.  29.  28. 307.  33. 175.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [16]:
import os
embeddings_index = {}
f = open( 'glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Number of word vectors is %s.' % len(embeddings_index))

Number of word vectors is 400000.


In [17]:
embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [18]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            50,
                            weights=[embedding_matrix],
                            input_length=25,
                            trainable=False)

In [19]:
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
#from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
def softmax(x, axis=1):
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [21]:
Tx = 25
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(25, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes = 1)

In [22]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator([a,s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context

In [23]:
post_activation_LSTM_cell = LSTM(128, return_state = True) # post-attention LSTM 
output_layer = Dense(20000, activation=softmax)

In [24]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):

    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    e = embedding_layer(X)
    a = Bidirectional(LSTM(units = n_a ,return_sequences = True))(e)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs = context,initial_state = [s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X,s0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [25]:
model = model(25,25,128,128,20000,20000)

In [26]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 25)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 25, 50)       64950       input_1[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 25, 256)      183296      embedding_1[0][0]                
____________________________________________________________________________________________

In [27]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [28]:
s = np.zeros((500, 128))
c = np.zeros((500, 128))

In [29]:
model.fit([context,s,c],response,epochs=5,batch_size=50)

Epoch 1/5
500/500 [==============================] - 28s 56ms/step - loss: 169.5479 - dense_3_loss: 6.3779 - dense_3_accuracy: 0.0500 - dense_3_accuracy_1: 0.1600 - dense_3_accuracy_2: 0.3280 - dense_3_accuracy_3: 0.4660 - dense_3_accuracy_4: 0.5780 - dense_3_accuracy_5: 0.6960 - dense_3_accuracy_6: 0.7480 - dense_3_accuracy_7: 0.7900 - dense_3_accuracy_8: 0.8180 - dense_3_accuracy_9: 0.8420 - dense_3_accuracy_10: 0.8600 - dense_3_accuracy_11: 0.8780 - dense_3_accuracy_12: 0.8900 - dense_3_accuracy_13: 0.8980 - dense_3_accuracy_14: 0.9000 - dense_3_accuracy_15: 0.9020 - dense_3_accuracy_16: 0.9040 - dense_3_accuracy_17: 0.9040 - dense_3_accuracy_18: 0.9040 - dense_3_accuracy_19: 0.9040 - dense_3_accuracy_20: 0.9040 - dense_3_accuracy_21: 0.9040 - dense_3_accuracy_22: 0.9040 - dense_3_accuracy_23: 0.9040 - dense_3_accuracy_24: 0.9040
Epoch 2/5
500/500 [==============================] - 11s 21ms/step - loss: 39.8680 - dense_3_loss: 0.2556 - dense_3_accuracy: 0.0500 - dense_3_accuracy_1: 

In [30]:
q = [("do you like sports")]

In [31]:
seq = tokenizer.texts_to_sequences(q)
p = pad_sequences(seq,maxlen=max_len, truncating="post",padding = 'post')

In [32]:
p

array([[58,  2, 46,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [33]:
pred = model.predict([p , np.zeros((1,128)) , np.zeros((1,128))])

In [34]:
pred

[array([[4.8473099e-01, 2.9661965e-07, 8.6087156e-03, ..., 2.9947341e-07,
         2.9087417e-07, 2.6201567e-07]], dtype=float32),
 array([[8.2227993e-01, 1.1567771e-08, 4.9335640e-03, ..., 1.1645096e-08,
         1.1287719e-08, 9.8676951e-09]], dtype=float32),
 array([[8.5536706e-01, 6.6496457e-09, 4.3089585e-03, ..., 6.6907639e-09,
         6.4943677e-09, 5.6580527e-09]], dtype=float32),
 array([[8.6041981e-01, 6.0722529e-09, 4.1951789e-03, ..., 6.1090906e-09,
         5.9410099e-09, 5.1797162e-09]], dtype=float32),
 array([[8.6194766e-01, 5.9210334e-09, 4.1510663e-03, ..., 5.9565899e-09,
         5.8033329e-09, 5.0657745e-09]], dtype=float32),
 array([[8.6291027e-01, 5.8333316e-09, 4.1195438e-03, ..., 5.8680594e-09,
         5.7265637e-09, 5.0045146e-09]], dtype=float32),
 array([[8.6369419e-01, 5.7637228e-09, 4.0932400e-03, ..., 5.7977818e-09,
         5.6661560e-09, 4.9567603e-09]], dtype=float32),
 array([[8.6435020e-01, 5.7059042e-09, 4.0711062e-03, ..., 5.7394138e-09,
         

In [35]:
import math
def beam_search_decoder(predictions, top_k = 3):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(20000 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [36]:
output = beam_search_decoder(pred)

In [37]:
output

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -4.252407861751897),
 ([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -8.28322680347056),
 ([6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -8.397724988905232)]

In [38]:
index_word[2]

'you'

In [39]:
index_word[6]

'the'